In [40]:
# import packages
import numpy as np
from numpy import random
from scipy import stats
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import PolynomialFeatures
import time
from sklearn.model_selection import KFold

In [2]:
data = pd.read_csv('/Users/arberimbibaj/dataset_example_indicatorCATE.csv', header=None, index_col=[0])
data = data.to_numpy()
data

array([[  4.907737,  -0.23949 ,   0.202046, ...,  -0.274776,   1.      ,
          8.      ],
       [-15.366406,   0.785973,  -0.935212, ...,   0.146119,   1.      ,
          0.      ],
       [  8.007676,   1.097277,  -0.962872, ...,  -0.791275,   1.      ,
          0.      ],
       ...,
       [ 14.967829,   1.437212,   0.055816, ...,   0.720359,   0.      ,
          0.      ],
       [  8.21161 ,  -0.568252,   0.788217, ...,   0.234121,   0.      ,
          8.      ],
       [ -4.206514,  -0.792531,   1.636563, ...,   2.661297,   0.      ,
          8.      ]])

In [3]:
N = len(data)
d = len(data[0,:]) - 3
print(N)
print(d)

1000
25


In [4]:
# train test split
random.shuffle(data)
training, test = data[:700,:], data[700:,:]

In [5]:
# slice dataset by treatment status
training_control = training[training[:,26]==0]
training_treatment = training[training[:,26]==1]

# Y_train by treatment status
Y_train_control = training_control[:,0]
Y_train_treatment = training_treatment[:,0]

# X_train by treatment status
X_train_control = training_control[:,1:26]
X_train_treatment = training_treatment[:,1:26]

# X and Y test
X_test = test[:,1:26]
Y_test = test[:,0]

# X_train and Y_train (no split by treatment status)
X_train = training[:,1:26]
Y_train = training[:,0]

# W_train and W_test
W_train = training[:,26]
W_test = test[:,26]

# tau_test
tau_test = test[:,27]

In [ ]:
X_train_control

In [ ]:
X_train_treatment

In [ ]:
Y_train_control

In [ ]:
Y_train_treatment

In [ ]:
X_train

In [ ]:
W_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
W_test

In [ ]:
Y_test

In [ ]:
tau_test

# T-Learner

In [ ]:
# T-Learner (example with Random Forest)

# mu_0
t_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu0.fit(X_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu1.fit(X_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_test)
# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
t_tau_hat

In [ ]:
# mean squared error
((t_tau_hat - tau_test)**2).mean()

# S-Learner

In [ ]:
### S-Learner

In [6]:
# set training and test features for the S-Learner (it views W as no different from other X's)
X_W_train = training[:,1:27]
X_test_0 = np.concatenate((test[:,1:26],np.zeros((300,1))), axis=1)
X_test_1 = np.concatenate((test[:,1:26],np.ones((300,1))), axis=1)

In [ ]:
X_test_0

In [ ]:
X_test_1

In [ ]:
Y_train

In [ ]:
# S-learner (example with Random Forest)

# mu_x
s_learner = RandomForestRegressor(max_depth=100, random_state=0)
s_learner.fit(X_W_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
s_tau_hat


In [ ]:
# mean squared error
((s_tau_hat - tau_test)**2).mean()

# X-Learner

In [ ]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_0_hat.fit(X_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_1_hat.fit(X_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = RandomForestClassifier(max_depth=100, random_state=0)
g_x_hat.fit(X_train,W_train)
probabilities = g_x_hat.predict_proba(X_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_test) + probas_0 * x_tau_1_hat.predict(X_test)
x_tau_hat


In [ ]:
# mean squared error (much lower here!)
((x_tau_hat - tau_test)**2).mean()

# R-Learner

In [ ]:
### R-Learner

# estimate e_x
r_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
r_learner_e_x.fit(X_train,W_train)

# get e_x predictions
r_probas = r_learner_e_x.predict_proba(X_train)
r_probas_0 = r_probas[:,0] # probabilities of W=0
r_probas_1 = r_probas[:,1] # probabilities of W=1

# estimate mu_x
r_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_mu_x.fit(X_train,Y_train)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - r_learner_mu_x.predict(X_train)) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_tau.fit(X_train,r_learner_pseudo_outcomes, sample_weight=r_learner_weights)

# predict tau
r_tau_hats = r_learner_tau.predict(X_test)
r_tau_hats

In [ ]:
((r_tau_hats - tau_test)**2).mean()

# DR-Learner

In [ ]:
### DR-Learner

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
dr_learner_e_x.fit(X_train, W_train)

dr_probas = dr_learner_e_x.predict_proba(X_train)
dr_probas_0 = dr_probas[:,0] # probabilities of W=0
dr_probas_1 = dr_probas[:,1] # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_mu_0.fit(X_train_control,Y_train_control)

# estimate mu_1
dr_learner_mu_1 = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_mu_1.fit(X_train_treatment,Y_train_treatment)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1.predict(X_train) + (1 - W_train) * dr_learner_mu_0.predict(X_train) # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1.predict(X_train) - dr_learner_mu_0.predict(X_train)

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_tau_hat.fit(X_train,dr_pseudo_outcomes)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_test)
dr_tau_hat

In [ ]:
((dr_tau_hat - tau_test)**2).mean()

# RA-Learner

In [ ]:
### RA-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
ra_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_train) - Y_train)

# tau_hat
ra_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
ra_tau_hat_learner.fit(X_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_test)
ra_tau_hat

In [ ]:
# mean squared error
((ra_tau_hat - tau_test)**2).mean()

# PW-Learner

In [ ]:
### PW-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
pw_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
pw_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_test)
pw_tau_hat

In [ ]:
# mean squared error
((pw_tau_hat - tau_test)**2).mean()

# F-Learner: AKA THE SAME AS PW-LEARNER!!!

In [ ]:
### F-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
f_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
f_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
f_pseudo_outcome = (W_train/f_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(f_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
f_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
f_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
f_tau_hat = f_tau_hat_learner.predict(X_test)
f_tau_hat

In [ ]:
# mean squared error
print(((f_tau_hat - tau_test)**2).mean())
print("Same as for PW-Learner")

# U-Learner

In [ ]:
### U-Learner
# estimate e_x
u_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
u_learner_e_x.fit(X_train,W_train)

# estimate mu_x
u_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
u_learner_mu_x.fit(X_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_train))/(W_train - u_learner_e_x.predict_proba(X_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
u_tau_hat_learner.fit(X_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_test)
u_tau_hats


In [ ]:
# mean squared error
((u_tau_hats - tau_test)**2).mean()

# Just some lasso tests

In [ ]:
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

In [ ]:
lasso_poly = LassoCV(cv=10, random_state=0, tol=1e-2)

In [ ]:
import time

t0 = time.time()
lasso_poly.fit(X_poly_train,Y_train)

t1 = time.time()

total = t1-t0
print(f"{total:.4f} seconds")

In [ ]:
# degree 3: 10 seconds to fit (100% cpu)
# degree 4: 90 seconds to fit (100% cpu)

In [ ]:
y_predictions = lasso_poly.predict(X_poly_test)

In [ ]:
((y_predictions - Y_test)**2).mean()

# just some Neural Network test

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model = keras.Sequential([
    keras.Input(shape=(d,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, start_from_epoch=0)

In [ ]:
print("Training Model")
training = model.fit(
    X_train,
    Y_train,
    batch_size=100,
    epochs=10,
    validation_data=(X_test, Y_test),
    callbacks=[callback] # include early stopping
)

In [ ]:
predictions

In [ ]:
print("Generate predictions for test samples")
predictions = np.reshape(model.predict(X_test),(300,))
print("predictions shape:", predictions.shape)
predictions

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=100)

In [ ]:
((predictions - Y_test)**2).mean()

# TRY SAME WITH LASSO AND NEURAL NETWORK!

## With Lasso (or L1-loss for logistic regression)

In [8]:
from sklearn.preprocessing import PolynomialFeatures
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

array([[ 1.423163  ,  0.029483  , -0.763018  , ...,  1.27407353,
         2.13154239,  3.5660995 ],
       [-1.50415   ,  0.967986  , -0.541778  , ...,  0.34996475,
        -0.17097233,  0.08352709],
       [-0.3594    ,  0.617325  , -0.377409  , ...,  0.21010694,
        -0.20856468,  0.20703374],
       ...,
       [ 0.643689  ,  0.137531  , -0.567255  , ...,  0.50074572,
         0.32062984,  0.20530079],
       [-0.356429  ,  1.838532  , -2.410945  , ...,  0.01240463,
         0.06255419,  0.31544884],
       [-1.148046  ,  0.419502  ,  0.037405  , ...,  0.17600748,
        -0.06986855,  0.02773527]])

In [15]:
# compute polynomial features for treatment and control groups in training set
poly_train_treatment = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train_treatment = poly_train_treatment.fit_transform(X_train_treatment)
poly_train_control = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train_control = poly_train_treatment.fit_transform(X_train_control)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)

In [10]:
X_poly_train_control

array([[-3.59400000e-01,  6.17325000e-01, -3.77409000e-01, ...,
         2.10106938e-01, -2.08564680e-01,  2.07033743e-01],
       [-4.64694000e-01, -2.82128000e-01,  7.95796000e-01, ...,
         3.76589191e-01,  4.36669135e-01,  5.06334058e-01],
       [-1.48957600e+00,  2.42270000e-02,  2.87398000e-01, ...,
         1.75615971e-01,  4.81072377e-01,  1.31782224e+00],
       ...,
       [-3.63758000e-01,  1.28144500e+00, -1.16883500e+00, ...,
         3.70097512e-01, -6.38204485e-01,  1.10053419e+00],
       [ 2.07388000e+00, -3.06304500e+00,  2.66972200e+00, ...,
         1.57479600e+01, -1.62172358e+01,  1.67004955e+01],
       [-3.56429000e-01,  1.83853200e+00, -2.41094500e+00, ...,
         1.24046328e-02,  6.25541926e-02,  3.15448838e-01]])

In [11]:
X_poly_train_treatment


array([[ 1.423163  ,  0.029483  , -0.763018  , ...,  1.27407353,
         2.13154239,  3.5660995 ],
       [-1.50415   ,  0.967986  , -0.541778  , ...,  0.34996475,
        -0.17097233,  0.08352709],
       [ 0.253921  ,  0.773891  , -1.295039  , ...,  0.8274789 ,
         0.92029438,  1.02352066],
       ...,
       [-1.195314  ,  1.512897  , -1.441811  , ...,  0.64155119,
        -1.29486699,  2.61347894],
       [ 0.643689  ,  0.137531  , -0.567255  , ...,  0.50074572,
         0.32062984,  0.20530079],
       [-1.148046  ,  0.419502  ,  0.037405  , ...,  0.17600748,
        -0.06986855,  0.02773527]])

## T-learner Lasso

In [19]:

# T-Learner (example with Lasso)
tic = time.perf_counter()
# mu_0
t_learner_mu0 = LassoCV(cv=10, tol=1e-2, random_state=0)
t_learner_mu0.fit(X_poly_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_poly_test)

# mu_1
t_learner_mu1 = LassoCV(cv=10, tol=1e-2, random_state=0)
t_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_poly_test)

# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
toc = time.perf_counter()
t_tau_hat

array([-2.29218340e+00,  5.00908032e+00,  5.23536753e+00, -8.36889645e-02,
       -1.05221149e+00, -2.72051717e-01, -1.04028329e-01,  1.98170726e+00,
        1.13551254e+01,  1.34619102e+00,  3.80487589e+00,  5.15734053e+00,
        9.61243856e+00,  6.15261006e+00,  6.06190221e+00,  1.01536992e+00,
        4.10380480e+00, -7.28702488e-01,  4.00465252e+00, -2.43254604e+00,
        9.26057685e+00,  2.53865219e+00,  5.99777413e-01, -2.23093941e+00,
       -6.59069208e-01,  7.23894133e+00,  7.91543298e+00,  8.60687305e-01,
        8.83559552e+00,  4.71083043e+00,  9.42553238e+00,  4.05838011e-01,
        8.20059020e+00,  8.03119550e+00,  3.12076124e-01,  2.98185001e+00,
        7.46251605e-01,  9.23639733e+00,  1.36506917e+00,  3.57460919e+00,
        5.22689331e+00,  2.43599950e+00,  7.30069697e-01,  1.26278202e+00,
       -1.39611641e+00, -2.29825936e+00, -5.03671104e-01,  5.51734674e+00,
        5.28482151e+00,  2.86900126e+00,  8.02955298e+00,  7.33189681e+00,
        5.17974029e+00, -

In [24]:
print(f'Time for calculations: {toc-tic}') # took 69 seconds

Time for calculations: 69.58790112499992


In [25]:
((t_tau_hat - tau_test)**2).mean()

6.182381044567873

## S-leaerner Lasso

In [30]:
# compute polynomial features for treatment and control groups in training set
xw_poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_W_poly_train = poly_train_treatment.fit_transform(X_W_train)

xw_poly_test_0 = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test_0 = xw_poly_test_0.fit_transform(X_test_0)

xw_poly_test_1 = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test_1 = xw_poly_test_0.fit_transform(X_test_1)

In [39]:
# S-learner (example with Random Forest)
tic = time.perf_counter()
# mu_x
s_learner = LassoCV(cv=10, tol=1e-2, random_state=0)
s_learner.fit(X_W_poly_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_poly_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_poly_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
toc = time.perf_counter()
print(f'Time for computation: {toc-tic}')

Time for computation; 69.74598945799971


In [38]:
((s_tau_hat - tau_test)**2).mean()

4.432630658397295

## X-learner with lasso (or l1-penalty)
### TAKES A LOT OF TIME!

In [45]:
### X-Learner

tic = time.perf_counter()

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = LassoCV(cv=10, tol=1, random_state=0)
x_learner_mu0.fit(X_poly_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = LassoCV(cv=10, tol=1, random_state=0)
x_learner_mu1.fit(X_poly_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_poly_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_poly_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = LassoCV(cv=10, tol=1, random_state=0)
x_tau_0_hat.fit(X_poly_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = LassoCV(cv=10, tol=1, random_state=0)
x_tau_1_hat.fit(X_poly_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = LogisticRegressionCV(cv=KFold(10), penalty='l1', tol=1, random_state=0)
g_x_hat.fit(X_poly_train,W_train)
probabilities = g_x_hat.predict_proba(X_poly_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_poly_test) + probas_0 * x_tau_1_hat.predict(X_poly_test)

toc = time.perf_counter()

print(f'Time for computation: {toc-tic}')

KeyboardInterrupt: 

In [ ]:
((x_tau_hat - tau_test)**2).mean()